In [1]:
SUBJECT_COUNT = 32 #Num of objects in dataset - FAMED: 32, YouTubeCelenrities :47
LATEST_CHECKPOINT_PATH = 'KL/latest/checkpoint.h5'
BEST_CHECKPOINT_PATH = 'KL/best/weights.loss-{val_loss:.2f}.h5'
MODEL_PATH_TRAIN = 'KL/model_famed/model10.h5'
MODEL_PATH_SAVE = 'KL/model.h5'

# Train setting
BATCH_SIZE = 128
EPOCH_EACH_TRAIN = 5
TRAIN_FROM = 'MODEL' # Possible options: 'BEST' | 'LATEST' | 'MODEL' | 'NEW'

In [3]:
import re
import os
import cv2
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import initializers
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten,Activation,BatchNormalization 
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import tensorflow.keras.backend as K

In [ ]:
g_config = None
with open('KL/config_FAMED.json', 'r+') as f_config: #change file "config.json" if train on YouTube Celebrities dataset
    g_config = json.loads(f_config.read())

print(g_config)

In [4]:
def get_subject_from_filename(filename):
    #r = re.search("\d\d\d\d_\d\d_\d\d\d_(\w+_\w+)_\w+_\w+.jpg", filename)  #use this for train on YTC dataset
    r = re.search("(\d+)_.+\.jpg", filename)
    label=r.group(1)
    return label

def get_label_from_filename(filename):
    return g_config['labels'][get_subject_from_filename(filename)]

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = keras.applications.imagenet_utils.preprocess_input(img)
    return img

In [5]:
class Custom_Dataset_Generator(keras.utils.Sequence):
    def __init__(self, folder, batch_size, shuffle):
        super().__init__()
        self.folder = folder
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.list_files = [ os.path.join(folder, file) for file in os.listdir(folder) ]        
        self.on_epoch_end()

    def __len__(self):
        return (np.ceil(len(self.list_files) / float(self.batch_size))).astype(np.int)

    def __getitem__(self, idx):
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]

        batch_x = [ preprocess_image(self.list_files[i]) for i in indexes ]
        batch_y = [ [ get_label_from_filename(self.list_files[i]) ] for i in indexes ]

        return np.array(batch_x), np.array(batch_y)
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_files))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
# From: https://github.com/CyberZHG/keras-lr-multiplier/blob/master/keras_lr_multiplier/multiplier.py
class LearningRateMultiplier(keras.optimizers.Optimizer):    
    def __init__(self,
                 optimizer,
                 multipliers,
                 **kwargs):
        """Initialize the optimizer wrapper.
        :param optimizer: The original optimizer.
        :param multipliers: A dict representing the multipliers.
                            The key is the prefix of the weight to be multiplied.
        :param kwargs: Arguments for parent class.
        """
        super(LearningRateMultiplier, self).__init__(**kwargs)
        self.optimizer = keras.optimizers.get(optimizer)
        self.multipliers = multipliers
        if hasattr(self.optimizer, 'learning_rate'):
            self.lr_attr = 'learning_rate'
        else:
            self.lr_attr = 'lr'

    @property
    def lr(self):
        return self.optimizer.lr

    @lr.setter
    def lr(self, lr):
        self.optimizer.lr = lr

    @property
    def learning_rate(self):
        return self.optimizer.learning_rate

    @learning_rate.setter
    def learning_rate(self, learning_rate):
        try:
            self.optimizer.learning_rate = learning_rate
        except ValueError:
            self.optimizer._hyper['learning_rate'] = learning_rate

    def _resource_apply_dense(self, *args, **kwargs):
        return self.optimizer._resource_apply_dense(*args, **kwargs)

    def _resource_apply_sparse(self, *args, **kwargs):
        return self.optimizer._resource_apply_sparse(*args, **kwargs)

    def _get_multiplier(self, name):
        multiplier, prefix_len = 1.0, 0
        for key, val in self.multipliers.items():
            if name.startswith(key):
                if len(key) > prefix_len:
                    prefix_len = len(key)
                    multiplier = val
        return multiplier

    def get_updates(self, loss, params):
        if len(self.updates) > 0:
            return self.updates
        multiplies = {}
        for param in params:
            multiplier = self._get_multiplier(param.name)
            if multiplier not in multiplies:
                multiplies[multiplier] = []
            multiplies[multiplier].append(param)

        self.updates, self.weights = [], []
        origin_lr = getattr(self, self.lr_attr)
        for i, (multiplier, params) in enumerate(multiplies.items()):
            lr = origin_lr
            if callable(multiplier):
                lr = lr * multiplier(K.cast(self.optimizer.iterations, K.floatx()))
            elif multiplier != 1.0:
                lr = lr * multiplier
            setattr(self, self.lr_attr, lr)
            with K.name_scope('Group_{}'.format(i)):
                self.updates += self.optimizer.get_updates(loss, params)
            print(self.multipliers, i, self.optimizer.weights)
            for w in self.optimizer.weights:
                if w not in self.weights:
                    self.weights.append(w)
        setattr(self, self.lr_attr, origin_lr)

        return self.updates

    def get_config(self):
        config = {
            'optimizer': keras.optimizers.serialize(self.optimizer),
            'multipliers': self.multipliers
        }
        base_config = super(LearningRateMultiplier, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        optimizer = keras.optimizers.deserialize(config.pop('optimizer'))
        return cls(optimizer, **config)    

In [7]:
vgg16_model = Sequential()

vgg16_model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))

vgg16_model.add(Convolution2D(64, (3, 3), activation='relu'))
vgg16_model.add(ZeroPadding2D((1,1)))
vgg16_model.add(Convolution2D(64, (3, 3), activation='relu'))
vgg16_model.add(MaxPooling2D((2,2), strides=(2,2)))

vgg16_model.add(ZeroPadding2D((1,1)))	
vgg16_model.add(Convolution2D(128, (3, 3), activation='relu'))
vgg16_model.add(ZeroPadding2D((1,1)))
vgg16_model.add(Convolution2D(128, (3, 3), activation='relu'))
vgg16_model.add(MaxPooling2D((2,2), strides=(2,2)))

vgg16_model.add(ZeroPadding2D((1,1)))
vgg16_model.add(Convolution2D(256, (3, 3), activation='relu'))
vgg16_model.add(ZeroPadding2D((1,1)))
vgg16_model.add(Convolution2D(256, (3, 3), activation='relu'))
vgg16_model.add(ZeroPadding2D((1,1)))
vgg16_model.add(Convolution2D(256, (3, 3), activation='relu'))
vgg16_model.add(MaxPooling2D((2,2), strides=(2,2)))

vgg16_model.add(ZeroPadding2D((1,1)))
vgg16_model.add(Convolution2D(512, (3, 3), activation='relu'))
vgg16_model.add(ZeroPadding2D((1,1)))
vgg16_model.add(Convolution2D(512, (3, 3), activation='relu'))
vgg16_model.add(ZeroPadding2D((1,1)))
vgg16_model.add(Convolution2D(512, (3, 3), activation='relu'))
vgg16_model.add(MaxPooling2D((2,2), strides=(2,2)))

vgg16_model.add(ZeroPadding2D((1,1)))
vgg16_model.add(Convolution2D(512, (3, 3), activation='relu'))
vgg16_model.add(ZeroPadding2D((1,1)))
vgg16_model.add(Convolution2D(512, (3, 3), activation='relu'))
vgg16_model.add(ZeroPadding2D((1,1)))
vgg16_model.add(Convolution2D(512, (3, 3), activation='relu'))
vgg16_model.add(MaxPooling2D((2,2), strides=(2,2)))

vgg16_model.add(Convolution2D(4096, (7, 7), activation='relu', name='fc6'))
vgg16_model.add(Dropout(0.5))

vgg16_model.add(Convolution2D(4096, (1, 1), activation='relu', name='fc7'))
vgg16_model.add(Dropout(0.5))

vgg16_model.add(Convolution2D(2622, (1, 1), name='fc8'))

vgg16_model.add(Flatten())
vgg16_model.add(Activation('softmax'))

# vgg16_model.load_weights('KL/db/vgg_face_weights.h5')

In [8]:
custom_layers = Dense(
    SUBJECT_COUNT, 
    name='custom_fc8',
    kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.01)    
)(vgg16_model.layers[-4].output)

custom_layers = Activation('softmax')(custom_layers)

custom_layers = Flatten()(custom_layers)

model = Model(inputs=vgg16_model.layers[0].input, outputs=custom_layers)
model.compile(
    optimizer=LearningRateMultiplier(
        keras.optimizers.SGD(learning_rate=0.001),
        { 'custom_fc8': 10 },
        name='LearningRateMultiplier'
    ),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [9]:
# model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_13_input (Inp [(None, 224, 224, 3)]     0         
_________________________________________________________________
zero_padding2d_13 (ZeroPaddi (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_14 (ZeroPaddi (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 114, 114, 64)      0     

In [10]:
# Checkpoint
latest_cp = tf.train.latest_checkpoint(os.path.dirname(LATEST_CHECKPOINT_PATH))
best_cp = os.path.join(
    os.path.dirname(BEST_CHECKPOINT_PATH), 
    next(filter(
        lambda f: f.endswith('.h5'),
        sorted(os.listdir(os.path.dirname(BEST_CHECKPOINT_PATH)))
    ))
)

if TRAIN_FROM == 'BEST':
    model.load_weights(best_cp)
    print("Train from BEST")
elif TRAIN_FROM == 'LATEST':
    model.load_weights(LATEST_CHECKPOINT_PATH) 
    print("Train from LATEST")
elif TRAIN_FROM == 'MODEL' and os.path.isfile(MODEL_PATH_TRAIN):
    model.load_weights(MODEL_PATH_TRAIN)
    print("Train from MODEL: {}".format(MODEL_PATH_TRAIN.split('/')[-1]))
else:
    print("Train from NEW")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'KL/best'

In [11]:
best_cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=BEST_CHECKPOINT_PATH, 
    verbose=1, 
    save_best_only=True,
    save_freq='epoch'
)

latest_cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=LATEST_CHECKPOINT_PATH, 
    verbose=1, 
    save_weights_only=True,
    save_freq=4 * BATCH_SIZE
)

def train(epochs=None):
    if epochs is None:
        epochs = 10

    model.fit(
        train_data,        
        epochs=epochs,
        #validation_data=validation_data,
        callbacks=[best_cp_callback, latest_cp_callback],
        verbose=1,
        batch_size=BATCH_SIZE,
        use_multiprocessing=True,
        workers=6
    )
    
    print('Saving full model to: {}'.format(MODEL_PATH_SAVE))
    model.save_weights(MODEL_PATH_SAVE)

In [ ]:
train(2)